## KMeans Clustering

In [1]:
from sklearn.preprocessing import scale
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd


In [2]:
iris = load_iris() ## array형태로 저장되어있음

iris_df = pd.DataFrame(data=iris.data, columns=["sepal_length","sepal_width","petal_length","petal_width"]) # 컬럼 이름 지정
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
kmeans = KMeans(n_clusters=3, max_iter=300, random_state=316) #3개의 클러스털 생성, 316번동안 모델링함
kmeans.fit(iris_df)# 모델생성 후 fit 시킴

C:\Users\SM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3, random_state=316)

In [4]:
print(kmeans.labels_) # 예측한값  라벨 확인

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 2 2 2
 2 2 1 1 2 2 2 2 1 2 1 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 1 2
 2 1]


In [5]:
iris_df["target"] = iris.target # 데이터에 저장되어있던 정답값
iris_df["cluster"] = kmeans.labels_
iris_result = iris_df.groupby(["target","cluster"])["sepal_length"].count() #target: 정답값, cluster:예측값 비교
print(iris_result)

target  cluster
0       0          50
1       1          48
        2           2
2       1          14
        2          36
Name: sepal_length, dtype: int64


### 군집화 평가

#### 실루엣분석(Silhouette analysis)

- 각 군집 간 거리가 얼마나 효율적으로 분리되어 있는지를 평가
- 실루엣 계수(silhouette coefficient): 개별 데이터의 군집화 집표
    - $a(i)$ : 해당 데이터 포인트와 같은 군집 내에 있는 다른 데이터 포인트와의 평균거리 
    - $b(i)$ : 해당 데이터 포인트가 속하지 않는 군집 중 가장 가까운 군집과의 평균거리
    $$ s(i) = \frac{b(i)-a(i)}{\max(a(i),b(i))}, \qquad - \leq s(i) \leq 1  $$
    - 1에 가까울수록 근처 군집과 멀리 떨어져 있음
    - 0에 가까울수록 근처 군집과 가까움
    - 음수는 다른 군집에 할당되어 있음

In [6]:
## 군집화 평가 - 실루엣 계수 사용
from sklearn.metrics import silhouette_samples, silhouette_score

In [7]:
iris_df["silhouette"] = silhouette_samples(iris.data, iris_df["cluster"])

In [8]:
ave_score = silhouette_score(iris.data,iris_df["cluster"])
print("SAS:{0:.4f}".format(ave_score))

SAS:0.5528


In [9]:
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,target,cluster,silhouette
0,5.1,3.5,1.4,0.2,0,0,0.852955
1,4.9,3.0,1.4,0.2,0,0,0.815495
2,4.7,3.2,1.3,0.2,0,0,0.829315
3,4.6,3.1,1.5,0.2,0,0,0.805014
4,5.0,3.6,1.4,0.2,0,0,0.849302


In [10]:
iris_df.groupby("cluster")["silhouette"].mean() # 그룹0은 잘 분류되었지만. 1과 2는 다소 잘 분류되었다고 볼 수 없음

cluster
0    0.798140
1    0.417320
2    0.451105
Name: silhouette, dtype: float64

## GMM(Gaussian Mixture Model)
$$f(x) = \sum_{k=1}^K w_k f_k (x), \qquad f_k = N(\mu_k , \sigma_k^2) $$

- 개별 정규분포의 평균과 분산 추정
- 각 데이터가 어떤 정규분포에 해당되는지에 대한 확률 추정

In [11]:
## 
from sklearn.mixture import GaussianMixture

In [12]:
gmm = GaussianMixture(n_components=3, random_state=316).fit(iris.data)
iris_df["gmm"] = gmm.predict(iris.data)
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,target,cluster,silhouette,gmm
0,5.1,3.5,1.4,0.2,0,0,0.852955,1
1,4.9,3.0,1.4,0.2,0,0,0.815495,1
2,4.7,3.2,1.3,0.2,0,0,0.829315,1
3,4.6,3.1,1.5,0.2,0,0,0.805014,1
4,5.0,3.6,1.4,0.2,0,0,0.849302,1


In [13]:
iris_result = iris_df.groupby(["target"])["gmm"].value_counts()
print(iris_result) ## target: 정답, gmm: 예측값 (그룹 명이 변경될 수도 있음) , 여기 아래 예시에선 1과 0클러스터를 이름을 바꿔서 분류했다고 볼 수 있음 -> 정확도 높음 

target  gmm
0       1      50
1       0      45
        2       5
2       2      50
Name: gmm, dtype: int64


## 기타 군집법

- 평균이동(mean shift) : from sklearn.cluster import MeanShift
- DBSCAN(density based spatial clustering of applications with noise): from sklearn.cluster import DBSCAN
